<a href="https://colab.research.google.com/github/akashrathour99/Image-Classification/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unrar


In [ ]:
# unzar the file imported from the google drive
!unrar x "/content/drive/MyDrive/afinal_project /Scanned.rar"

In [ ]:
!unrar x "/content/drive/MyDrive/afinal_project /Foot_print.rar"

In [ ]:
#some dependecies for model 
from skimage.io import imread, imshow
from google.colab import files
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
#these  2 helps in rotaing the image without loosing the data
import argparse
import imutils


In [ ]:
##theis the code for image enhencement which leads to more data loss but when we have dectyloscopic type data then this will work fine 
left_dir="Scanned"
# getting the names of the data for furthur process
left_items = sorted([x for x in os.listdir(left_dir) if x.endswith(".jpg")], key=lambda x: (x.split(".")[0]))
for imgg in left_items:
  black = (0,0,0)
  white = (255,255,255)
  threshold = (160,160,160)
  img = Image.open("Scanned"+"/"+imgg).convert("LA")
  pixels = img.getdata()
  newPixels = []
  for pixel in pixels:
      if pixel < threshold:
          newPixels.append(black)
      else:
          newPixels.append(white)
  newImg = Image.new("RGB",img.size)
  newImg.putdata(newPixels)
  #saving the data or rewriting the same image after enhencement
  newImg.save("Scanned"+"/"+imgg)

In [ ]:
# this functio gonna help me to get the name of the person for ordering the data to the right file
def check(img):
  ans=""
  ans=img[0]
  if(ord(img[1])-48>=0 and ord(img[1])-48<=9):
    ans+=img[1]
  return ans

In [ ]:
# this function gonna help for rotating the images or creating more data for traing
#and saving the data to the write folder
# the data will be splited in 25% for testing
def rot(img,image):
  t=1
  for angle in np.arange(0, 360, 30):
    rotated = imutils.rotate_bound(image, angle)
    #this line for storing data for training data
    path="Foot_print/training_data/person"+check(img)+"/"+str(t)+"_"+img
    cv2.imwrite(path,rotated)
    #this code is for generating testing data
    if(t%4==0):
      path="Foot_print/test_data/person"+check(img)+"/"+str(t)+"_"+img
      cv2.imwrite(path,rotated)
    
    t=t+1


In [ ]:
#this code gonna help in spliting the scanned data to left and right foot and also changing the data to gray scale


left_dir="Scanned"
left_items = sorted([x for x in os.listdir(left_dir) if x.endswith(".jpg")], key=lambda x: (x.split(".")[0]))
for img in left_items:
  image = imread("Scanned"+"/"+img,as_gray=True)
  ## this is the gray scale formula
  ## imgGray = 0.2989 * R + 0.5870 * G + 0.1140 * B 
  h, w,= image.shape
  half = w//2
  # left foot image
  left_part = image[:, :half]    
  #right foot image       
  right_part = image[:, half:]
  # called the rotate function for rotaing the images at specified angle
  rot(img[:2]+"-0-"+img[2:],left_part)
  rot(img[:2]+"-1-"+img[2:],right_part)

In [ ]:
# Specifying the folder where images are present
TrainingImagePath='Foot_print/training_data'
 
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)
 
test_datagen = ImageDataGenerator()
 
# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')
 
 
# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')
 
# Printing class labels for each face
test_set.class_indices

In [ ]:


TrainClasses=training_set.class_indices


ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

# Saving the footprint map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)

# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
print("Mapping of Footprint and its ID",ResultMap)

# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

In [ ]:

from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

'''Initializing the Convolutional Neural Network'''
classifier= Sequential()

''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels
'''
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))

'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))

'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))

'''# STEP--3 FLattening'''
classifier.add(Flatten())

'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))

classifier.add(Dense(OutputNeurons, activation='softmax'))

'''# Compiling the CNN'''
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

###########################################################
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
classifier.fit_generator(
                    training_set,
                    steps_per_epoch=len(training_set),
                    epochs=35,
                    validation_data=test_set,
                    validation_steps=len(test_set))

EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

In [35]:

import numpy as np
from keras.preprocessing import image

ImagePath='/content/Foot_print/test_data/person8/4_8--0-4.jpg'
test_image=image.load_img(ImagePath,target_size=(64, 64))
test_image=image.img_to_array(test_image)

test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)

print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

########################################
Prediction is:  person8
